# FINAL PROJECT SUBMISSION

- Student Name: Ian Korir
- Student Pase: Fulltime-Hybrid student
- Scheduled Project Review: 22/03/2024
- Instructor Name: Nikita Njoroge
- Blog Post: 

# Introduction
## Overview
Business Problem:
Viusasa new movie studio aims to establish itself as a leading player in the entertainment industry by producing high-quality and commercially successful movies. However, the studio faces the challenge of navigating the complexities of the movie market to ensure its productions resonate with audiences and generate significant revenue.


Problem Statement: How can Viusasa's new movie studio strategically optimize its production and release strategy to maximize box office success and audience satisfaction?

## Objectives
Key Objectives:
1.	Identify the most profitable and audience-friendly movie genres to prioritize in production efforts.
2.	Determine the optimal timing for movie releases to maximize audience turnout and box office revenue.
3.	Enhance audience engagement and satisfaction by aligning movie offerings with audience preferences and expectations.

## The Data
The datasets are stored in the 'zippedData' folder which were sourced from:
1.	Box Office MojoLinks : 
	bom.movie_gross.csv
2.	IMDBLinks : 
	im.db
3.	Rotten TomatoesLinks : 
	rt.movie_info.tsv
4.	TheMovieDBLinks : 
	tmdb.movies.csv
5.	The Numbers :
	tn.movie_budgets.csv




## Import relevant libraries


In [30]:
#importing libraries
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt

In [31]:
# read movie budget csv file
movie_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

# get summary of the dataframe
movie_budget.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


Check for missing values

In [32]:
# check for missing values
movie_budget.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

Clean data by changing data type to float

In [33]:
#Cast to numeric data type
movie_budget['production_budget'] = movie_budget['production_budget'].replace('[^.0-9]', '', regex=True).astype('float32')
movie_budget['domestic_gross'] = movie_budget['domestic_gross'].replace('[^.0-9]', '', regex=True).astype('float32')
movie_budget['worldwide_gross'] = movie_budget['worldwide_gross'].replace('[^.0-9]', '',regex = True).astype('float64')
movie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000.0,760507648.0,2.776345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063872.0,1.045664e+09
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762352.0,1.497624e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09


The release date is an object, in order to filter the years convert to date time.

In [34]:
#converting release data object to dateTime
movie_budget['release_date'] = pd.to_datetime(movie_budget['release_date'], errors='coerce')
movie_budget['year'] = movie_budget['release_date'].dt.strftime('%Y')
movie_budget.head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
0,1,2009-12-18,Avatar,425000000.0,760507648.0,2.776345e+09,2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063872.0,1.045664e+09,2011
2,3,2019-06-07,Dark Phoenix,350000000.0,42762352.0,1.497624e+08,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09,2017
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662208.0,2.053311e+09,2015
6,7,2018-04-27,Avengers: Infinity War,300000000.0,678815488.0,2.048134e+09,2018
7,8,2007-05-24,Pirates of the Caribbean: At Worldâs End,300000000.0,309420416.0,9.634204e+08,2007
8,9,2017-11-17,Justice League,300000000.0,229024288.0,6.559452e+08,2017
9,10,2015-11-06,Spectre,300000000.0,200074176.0,8.796209e+08,2015


Finding the percentage profits of the movies based on the production budget and the world wide gross.

In [35]:
# calculate the profits or loss for each movie
movie_budget['profit/loss'] = movie_budget['worldwide_gross'] - movie_budget['production_budget']
movie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,profit/loss
0,1,2009-12-18,Avatar,425000000.0,760507648.0,2.776345e+09,2009,2.351345e+09
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063872.0,1.045664e+09,2011,6.350639e+08
2,3,2019-06-07,Dark Phoenix,350000000.0,42762352.0,1.497624e+08,2019,-2.002376e+08
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005856.0,1.403014e+09,2015,1.072414e+09
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181376.0,1.316722e+09,2017,9.997217e+08
